In [236]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime as dt

In [190]:
# Load the dataset
df_rain = pd.read_csv("weatherAUS.csv")


In [191]:
# Replace numerical columns with median
def replace_numerical(df):
    for col in df.select_dtypes(['int', 'float']):
        df[col] = df[col].fillna(df[col].median())
    return df

# Replace object columns with mode
def replace_object(df):
    for col in df.select_dtypes('object'):
        df[col] = df[col].fillna(method='ffill')
    return df

In [192]:
df = df_rain.copy()
df = replace_numerical(df)
df = replace_object(df)
df['RainToday']=df['RainToday'].map({'Yes':1,'No':0})

In [193]:
df['RainTomorrow']=df['RainTomorrow'].map({'Yes':1,'No':0})

In [5]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,4.8,8.4,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,5.0,16.9,21.8,0,No
1,2008-12-02,Albury,7.4,25.1,0.0,4.8,8.4,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,5.0,5.0,17.2,24.3,0,No
2,2008-12-03,Albury,12.9,25.7,0.0,4.8,8.4,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,0,No
3,2008-12-04,Albury,9.2,28.0,0.0,4.8,8.4,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,5.0,5.0,18.1,26.5,0,No
4,2008-12-05,Albury,17.5,32.3,1.0,4.8,8.4,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,No


# NO/LIGHT/MEDIUM/HIGH

In [12]:
temp = np.arange(-10, 51, 1)
humidity = np.arange(0, 101, 1)
pressure = np.arange(980, 1061, 1)
rainfall = np.arange(0, 101, 1)
rain = np.arange(0, 100, 1)

rainfall_level = np.arange(0, 101, 1)
rainfall_none = fuzz.trimf(rainfall_level, [0, 0, 10])
rainfall_light = fuzz.trimf(rainfall_level, [0, 10, 30])
rainfall_medium = fuzz.trimf(rainfall_level, [10, 30, 60])
rainfall_heavy = fuzz.trimf(rainfall_level, [30, 100, 100])

temp = np.arange(-10, 41, 1)
temp_cold = fuzz.trimf(temp, [-10, -10, 10])
temp_mild = fuzz.trimf(temp, [-10, 20, 30])
temp_hot = fuzz.trimf(temp, [20, 40, 40])

humidity = np.arange(0, 101, 1)
humidity_low = fuzz.trimf(humidity, [0, 0, 50])
humidity_medium = fuzz.trimf(humidity, [0, 50, 100])
humidity_high = fuzz.trimf(humidity, [50, 100, 100])

pressure = np.arange(980, 1061, 1)
pressure_low = fuzz.trimf(pressure, [980, 980, 1000])
pressure_medium = fuzz.trimf(pressure, [980, 1010, 1040])
pressure_high = fuzz.trimf(pressure, [1010, 1060, 1060])

rain_tomorrow_none = fuzz.trimf(rain, [0, 0, 5])
rain_tomorrow_light = fuzz.trimf(rain, [0, 20, 45])
rain_tomorrow_medium = fuzz.trimf(rain, [25, 50, 80])
rain_tomorrow_heavy = fuzz.trimf(rain, [70, 100, 100])


# Predict "raintomorrow" using the fuzzy system and linguistic variables
fuzzy_rainfall_none = fuzz.interp_membership(rainfall_level, rainfall_none, 40)
fuzzy_rainfall_light = fuzz.interp_membership(rainfall_level, rainfall_light, 40)
fuzzy_rainfall_medium = fuzz.interp_membership(rainfall_level, rainfall_medium, 40)
fuzzy_rainfall_heavy = fuzz.interp_membership(rainfall_level, rainfall_heavy, 40)
fuzzy_temp_cold = fuzz.interp_membership(temp, temp_cold, 40)
fuzzy_temp_mild = fuzz.interp_membership(temp, temp_mild, 40)
fuzzy_temp_hot = fuzz.interp_membership(temp, temp_hot, 40)
fuzzy_humidity_low = fuzz.interp_membership(humidity, humidity_low, 49)
fuzzy_humidity_medium = fuzz.interp_membership(humidity, humidity_medium, 49)
fuzzy_humidity_high = fuzz.interp_membership(humidity, humidity_high, 49)
fuzzy_pressure_low = fuzz.interp_membership(pressure, pressure_low, 1010)
fuzzy_pressure_medium = fuzz.interp_membership(pressure, pressure_medium, 1010)
fuzzy_pressure_high = fuzz.interp_membership(pressure, pressure_high, 1010)

rule1 = np.fmax(fuzzy_temp_cold, fuzzy_pressure_low)
rule2 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_none, fuzzy_rainfall_light), fuzzy_temp_cold), np.fmin(np.fmax(fuzzy_pressure_medium,fuzzy_pressure_high), np.fmax(fuzzy_humidity_low, fuzzy_humidity_medium)))
rule3 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_none, fuzzy_rainfall_light), fuzzy_temp_hot), np.fmin(fuzzy_pressure_high, np.fmax(fuzzy_humidity_low,fuzzy_humidity_medium)))

rain_activation_none1 = np.fmin(rain_tomorrow_none, rule1)
    
rain_activation_none2 = np.fmin(rain_tomorrow_none, rule2)
rain_activation_none3 = np.fmin(rain_tomorrow_none, rule3)
rain_activation_none = np.fmin(rain_activation_none1, rain_activation_none1, rain_activation_none1)

rule4 = np.fmin(fuzzy_temp_mild, np.fmax(fuzzy_humidity_high,fuzzy_rainfall_medium))
rule5 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_none,fuzzy_rainfall_light), fuzzy_temp_hot), np.fmin(fuzzy_pressure_low, np.fmax(fuzzy_humidity_low,fuzzy_humidity_medium)))
rain_activation_light1 = np.fmin(rain_tomorrow_light, rule4)
rain_activation_light2 = np.fmin(rain_tomorrow_light, rule5)
rain_activation_light = np.fmin(rain_activation_light2, rain_activation_light2, rain_activation_light2)

rule6 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_light, fuzzy_rainfall_medium), fuzzy_temp_mild), np.fmin(fuzzy_pressure_medium, np.fmax(fuzzy_humidity_high,fuzzy_humidity_medium)))
rain_activation_medium = np.fmin(rain_tomorrow_medium, rule6)

rule7 = np.fmin(fuzzy_temp_hot, np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_humidity_medium), fuzzy_pressure_medium))
rain_activation_heavy = np.fmin(rain_tomorrow_heavy, rule7)
aggregated = np.fmax(np.fmax(rain_activation_none, rain_activation_light), np.fmax(rain_activation_medium, rain_activation_heavy))

rain = fuzz.defuzz(rain, aggregated, 'centroid')

In [6]:
def prediction(rainfall_level_m, temp_m, humidity_m, pressure_m):

    temp = np.arange(-10, 51, 1)
    humidity = np.arange(0, 101, 1)
    pressure = np.arange(980, 1061, 1)
    rainfall = np.arange(0, 101, 1)
    rain = np.arange(0, 100, 1)

    rainfall_level = np.arange(-1, 101, 1)
    rainfall_none = fuzz.trimf(rainfall_level, [0, 0.5, 10])
    rainfall_light = fuzz.trimf(rainfall_level, [0.5, 10, 30])
    rainfall_medium = fuzz.trimf(rainfall_level, [10, 30, 60])
    rainfall_heavy = fuzz.trimf(rainfall_level, [30, 100, 100])

    temp = np.arange(-10, 41, 1)
    temp_cold = fuzz.trimf(temp, [-10, -10, 10])
    temp_mild = fuzz.trimf(temp, [-10, 20, 30])
    temp_hot = fuzz.trimf(temp, [20, 40, 40])

    humidity = np.arange(0, 101, 1)
    humidity_low = fuzz.trimf(humidity, [0, 0, 50])
    humidity_medium = fuzz.trimf(humidity, [0, 50, 100])
    humidity_high = fuzz.trimf(humidity, [50, 100, 100])

    pressure = np.arange(980, 1061, 1)
    pressure_low = fuzz.trimf(pressure, [980, 980, 1000])
    pressure_medium = fuzz.trimf(pressure, [980, 1010, 1040])
    pressure_high = fuzz.trimf(pressure, [1010, 1060, 1060])

    rain_tomorrow_none = fuzz.trimf(rain, [0, 0, 5])
    rain_tomorrow_light = fuzz.trimf(rain, [0, 20, 45])
    rain_tomorrow_medium = fuzz.trimf(rain, [25, 50, 80])
    rain_tomorrow_heavy = fuzz.trimf(rain, [70, 100, 100])

    # Predict "raintomorrow" using the fuzzy system and linguistic variables
    fuzzy_rainfall_none = fuzz.interp_membership(rainfall_level, rainfall_none, rainfall_level_m)
    fuzzy_rainfall_light = fuzz.interp_membership(rainfall_level, rainfall_light, rainfall_level_m)
    fuzzy_rainfall_medium = fuzz.interp_membership(rainfall_level, rainfall_medium, rainfall_level_m)
    fuzzy_rainfall_heavy = fuzz.interp_membership(rainfall_level, rainfall_heavy, rainfall_level_m)
    fuzzy_temp_cold = fuzz.interp_membership(temp, temp_cold, temp_m)
    fuzzy_temp_mild = fuzz.interp_membership(temp, temp_mild, temp_m)
    fuzzy_temp_hot = fuzz.interp_membership(temp, temp_hot, temp_m)
    fuzzy_humidity_low = fuzz.interp_membership(humidity, humidity_low, humidity_m)
    fuzzy_humidity_medium = fuzz.interp_membership(humidity, humidity_medium, humidity_m)
    fuzzy_humidity_high = fuzz.interp_membership(humidity, humidity_high, humidity_m)
    fuzzy_pressure_low = fuzz.interp_membership(pressure, pressure_low, pressure_m)
    fuzzy_pressure_medium = fuzz.interp_membership(pressure, pressure_medium, pressure_m)
    fuzzy_pressure_high = fuzz.interp_membership(pressure, pressure_high, pressure_m)

    rule1 = np.fmax(fuzzy_temp_cold, fuzzy_pressure_low)
    rule2 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_none, fuzzy_rainfall_light), fuzzy_temp_cold), np.fmin(np.fmax(fuzzy_pressure_medium,fuzzy_pressure_high), np.fmax(fuzzy_humidity_low, fuzzy_humidity_medium)))
    rule3 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_none, fuzzy_rainfall_light), fuzzy_temp_hot), np.fmin(fuzzy_pressure_high, np.fmax(fuzzy_humidity_low,fuzzy_humidity_medium)))

    rain_activation_none1 = np.fmin(rain_tomorrow_none, rule1)

    rain_activation_none2 = np.fmin(rain_tomorrow_none, rule2)
    rain_activation_none3 = np.fmin(rain_tomorrow_none, rule3)
    rain_activation_none = np.fmin(rain_activation_none1, rain_activation_none1, rain_activation_none1)

    rule4 = np.fmin(fuzzy_temp_mild, np.fmax(fuzzy_humidity_high,fuzzy_rainfall_medium))
    rule5 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_none,fuzzy_rainfall_light), fuzzy_temp_hot), np.fmin(fuzzy_pressure_low, np.fmax(fuzzy_humidity_low,fuzzy_humidity_medium)))

    rain_activation_light1 = np.fmin(rain_tomorrow_light, rule4)
    rain_activation_light2 = np.fmin(rain_tomorrow_light, rule5)
    rain_activation_light = np.fmin(rain_activation_light2, rain_activation_light2, rain_activation_light2)

    rule6 = np.fmin(np.fmin(np.fmax(fuzzy_rainfall_light, fuzzy_rainfall_medium), fuzzy_temp_mild), np.fmin(np.fmax(fuzzy_pressure_medium, fuzzy_pressure_high), np.fmax(fuzzy_humidity_high,fuzzy_humidity_medium)))
    #rule8 = np.fmin(fuzzy_temp_mild, np.fmin(np.fmax(fuzzy_humidity_high,fuzzy_humidity_medium), fuzzy_pressure_high))
    rain_activation_medium1 = np.fmin(rain_tomorrow_medium, rule6)
    #rain_activation_medium2 = np.fmin(rain_tomorrow_medium, rule8)
    #rain_activation_medium = np.fmin(rain_activation_medium1, rain_activation_medium2)
    
    rule7 = np.fmin(fuzzy_temp_hot, np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_humidity_medium), fuzzy_pressure_medium))
    rain_activation_heavy = np.fmin(rain_tomorrow_heavy, rule7)
    aggregated = np.fmax(np.fmax(rain_activation_none, rain_activation_light), np.fmax(rain_activation_medium1, rain_activation_heavy))
    #print(type(aggregated))
    if aggregated.mean() == 0:
        return -10
    else:
        rain = fuzz.defuzz(rain, aggregated, 'bisector')
        return rain

In [237]:
X = df.drop(['RainToday'], axis=1)
y = df['RainToday']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,4.8,8.4,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,5.0,16.9,21.8,0,0
1,2008-12-02,Albury,7.4,25.1,0.0,4.8,8.4,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,5.0,5.0,17.2,24.3,0,0
2,2008-12-03,Albury,12.9,25.7,0.0,4.8,8.4,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,0,0
3,2008-12-04,Albury,9.2,28.0,0.0,4.8,8.4,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,5.0,5.0,18.1,26.5,0,0
4,2008-12-05,Albury,17.5,32.3,1.0,4.8,8.4,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0


In [ ]:
predictions = []
pred_bin = []
for index, row in X_test.iterrows():
    #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
    pred = prediction(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
    if pred >= 50:
        pred_bin.append(1)
    else:
        pred_bin.append(0)
    predictions.append(pred)


In [15]:
pred = []
for i in predictions:
    if i >= 95:
        pred = pred+[1]
    else:
        pred = pred+[0]
yes_l = 0
for i in range(0,len(list(y_test)), 1):
    if list(y_test)[i] == pred[i]:
        yes_l += 1
print(yes_l/len(list(y_test))*100)

77.36834868692424


# YES/NO Triangle

In [225]:
def prediction(rainfall_level_m, temp_m, humidity_m, pressure_m, defu):

    temp = np.arange(-10, 51, 1)
    humidity = np.arange(0, 101, 1)
    pressure = np.arange(980, 1061, 1)
    rainfall = np.arange(0, 101, 1)
    rain_tomorrow = np.arange(0, 1.1, 0.1)

    rainfall_level = np.arange(0, 351, 1)
    rainfall_none = fuzz.trimf(rainfall_level, [0, 0.1, 0.5])
    rainfall_light = fuzz.trimf(rainfall_level, [0, 75, 180])
    rainfall_medium = fuzz.trimf(rainfall_level, [0.5, 180, 350])
    rainfall_heavy = fuzz.trimf(rainfall_level, [180, 350, 350])


    temp = np.arange(-10, 41, 1)
    temp_cold = fuzz.trimf(temp, [-10, -10, 15])
    temp_mild = fuzz.trimf(temp, [-10, 15, 40])
    temp_hot = fuzz.trimf(temp, [15, 40, 40])

    humidity = np.arange(0, 101, 1)
    humidity_low = fuzz.trimf(humidity, [0, 0, 50])
    humidity_medium = fuzz.trimf(humidity, [0, 50, 100])
    humidity_high = fuzz.trimf(humidity, [50, 100, 100])

    pressure = np.arange(980, 1061, 1)
    pressure_low = fuzz.trimf(pressure, [980, 980, 1020])
    pressure_medium = fuzz.trimf(pressure, [980, 1020, 1060])
    pressure_high = fuzz.trimf(pressure, [1020, 1060, 1060])

    rain_tomorrow_no = fuzz.trimf(rain_tomorrow, [0, 0, 0.5])
    rain_tomorrow_yes =  fuzz.trimf(rain_tomorrow, [0.5, 1, 1])

    # Predict "raintomorrow" using the fuzzy system and linguistic variables
    fuzzy_rainfall_none = fuzz.interp_membership(rainfall_level, rainfall_none, rainfall_level_m)
    fuzzy_rainfall_light = fuzz.interp_membership(rainfall_level, rainfall_light, rainfall_level_m)
    fuzzy_rainfall_medium = fuzz.interp_membership(rainfall_level, rainfall_medium, rainfall_level_m)
    fuzzy_rainfall_heavy = fuzz.interp_membership(rainfall_level, rainfall_heavy, rainfall_level_m)
    fuzzy_temp_cold = fuzz.interp_membership(temp, temp_cold, temp_m)
    fuzzy_temp_mild = fuzz.interp_membership(temp, temp_mild, temp_m)
    fuzzy_temp_hot = fuzz.interp_membership(temp, temp_hot, temp_m)
    fuzzy_humidity_low = fuzz.interp_membership(humidity, humidity_low, humidity_m)
    fuzzy_humidity_medium = fuzz.interp_membership(humidity, humidity_medium, humidity_m)
    fuzzy_humidity_high = fuzz.interp_membership(humidity, humidity_high, humidity_m)
    fuzzy_pressure_low = fuzz.interp_membership(pressure, pressure_low, pressure_m)
    fuzzy_pressure_medium = fuzz.interp_membership(pressure, pressure_medium, pressure_m)
    fuzzy_pressure_high = fuzz.interp_membership(pressure, pressure_high, pressure_m)

    rule1 = np.fmax(np.fmax(fuzzy_rainfall_none, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule2 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule3 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule4 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule5 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule6 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule7 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule8 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule9 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium ))
    rain_activation_no1 = np.fmin(rain_tomorrow_no, rule1)
    rain_activation_no2 = np.fmin(rain_tomorrow_no, rule2)
    rain_activation_no3 = np.fmin(rain_tomorrow_no, rule3)
    rain_activation_no4 = np.fmin(rain_tomorrow_no, rule4)
    rain_activation_no5 = np.fmin(rain_tomorrow_no, rule5)
    rain_activation_no6 = np.fmin(rain_tomorrow_no, rule6)
    rain_activation_no7 = np.fmin(rain_tomorrow_no, rule7)
    rain_activation_no8 = np.fmin(rain_tomorrow_no, rule8)
    rain_activation_no9 = np.fmin(rain_tomorrow_no, rule9)

    rain_activation_no = np.fmin(np.fmin(np.fmin(np.fmin(rain_activation_no1, rain_activation_no2), np.fmin(rain_activation_no3, rain_activation_no4)),
                        np.fmin(np.fmin(rain_activation_no5, rain_activation_no6), np.fmin(rain_activation_no7, rain_activation_no8))),
                        rain_activation_no9)

    rule10 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule11 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule12 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule13 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule14 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule15 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule16 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule17 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule18 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule19 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule20 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule21 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule22 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule23 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule24 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule25 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule26 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule27 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule28 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule29 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule30 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule31 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule32 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule33 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule34 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))

    rain_activation_yes = np.fmin(np.fmin(np.fmin(np.fmin(np.fmin(rule10, rule11), np.fmin(rule12, rule13)),
                                    np.fmin(np.fmin(rule14, rule15), np.fmin(rule16, rule17))),
                            np.fmin(np.fmin(np.fmin(rule18, rule19), np.fmin(rule20, rule21)),
                                    np.fmin(np.fmin(rule22, rule23), np.fmin(rule24, rule25)))),
                            np.fmin(np.fmin(np.fmin(np.fmin(rule26, rule27), np.fmin(rule28, rule29)),
                                    np.fmin(np.fmin(rule30, rule31), np.fmin(rule32, rule33))),
                            rule34))
    aggregated = np.fmax(rain_activation_no, rain_activation_yes)
    #print(aggregated)
    if aggregated.mean() == 0:
            #print('======================================================')
            return -10
    else:
            rain = fuzz.defuzz(rain_tomorrow, aggregated, defu)
            return rain

In [ ]:
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    start = dt.now()
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print('time = ', dt.now() - start)
    print(len(predictions))
    print('pred count 0 = ', predictions.count(0))
    print('pred count -10 = ', predictions.count(-10))
    pred = []
    for i in predictions:
        if i > 0.5:
            pred.append(1)
        elif i == -10:
            pred.append(3)
        else:
            pred.append(0)
    yes_l = 0
    for i in range(0,len(list(y_test)), 1):
        if list(y_test)[i] == pred[i]:
            yes_l += 1
    print(f, '>0.5 = ', yes_l/(len(list(y_test))-predictions.count(-10))*100)    


# Yes/No Trapezoid 

In [220]:
def prediction_trapezoid(rainfall_level_m, temp_m, humidity_m, pressure_m, defu):

    rainfall_level = np.arange(0, 351, 1)
    rainfall_none = fuzz.trapmf(rainfall_level, [0, 0, 1, 5])
    rainfall_light = fuzz.trapmf(rainfall_level, [1, 59, 117, 175])
    rainfall_medium = fuzz.trapmf(rainfall_level, [117, 175, 233, 291])
    rainfall_heavy = fuzz.trapmf(rainfall_level, [233, 291, 350, 350])

    temp = np.arange(-10, 41, 1)
    temp_cold = fuzz.trapmf(temp, [-10, -10, 0, 10])
    temp_mild = fuzz.trapmf(temp, [0, 10, 20, 30])
    temp_hot = fuzz.trapmf(temp, [20, 30, 40, 40])

    humidity = np.arange(0, 101, 1)
    humidity_low = fuzz.trapmf(humidity, [0, 0, 20, 40])
    humidity_medium = fuzz.trapmf(humidity, [20, 40, 60, 80])
    humidity_high = fuzz.trapmf(humidity, [60, 80, 100, 100])

    pressure = np.arange(980, 1061, 1)
    pressure_low = fuzz.trapmf(pressure, [980, 980,  996, 1012])
    pressure_medium = fuzz.trapmf(pressure, [996, 1012, 1028, 1044])
    pressure_high = fuzz.trapmf(pressure, [1028, 1044, 1060, 1060])

    rain_tomorrow = np.arange(0, 1.1, 0.1)
    rain_tomorrow_no = fuzz.trimf(rain_tomorrow, [0, 0, 0.5])
    rain_tomorrow_yes =  fuzz.trimf(rain_tomorrow, [0.5, 1, 1])

    # Predict "raintomorrow" using the fuzzy system and linguistic variables
    fuzzy_rainfall_none = fuzz.interp_membership(rainfall_level, rainfall_none, rainfall_level_m)
    fuzzy_rainfall_light = fuzz.interp_membership(rainfall_level, rainfall_light, rainfall_level_m)
    fuzzy_rainfall_medium = fuzz.interp_membership(rainfall_level, rainfall_medium, rainfall_level_m)
    fuzzy_rainfall_heavy = fuzz.interp_membership(rainfall_level, rainfall_heavy, rainfall_level_m)
    fuzzy_temp_cold = fuzz.interp_membership(temp, temp_cold, temp_m)
    fuzzy_temp_mild = fuzz.interp_membership(temp, temp_mild, temp_m)
    fuzzy_temp_hot = fuzz.interp_membership(temp, temp_hot, temp_m)
    fuzzy_humidity_low = fuzz.interp_membership(humidity, humidity_low, humidity_m)
    fuzzy_humidity_medium = fuzz.interp_membership(humidity, humidity_medium, humidity_m)
    fuzzy_humidity_high = fuzz.interp_membership(humidity, humidity_high, humidity_m)
    fuzzy_pressure_low = fuzz.interp_membership(pressure, pressure_low, pressure_m)
    fuzzy_pressure_medium = fuzz.interp_membership(pressure, pressure_medium, pressure_m)
    fuzzy_pressure_high = fuzz.interp_membership(pressure, pressure_high, pressure_m)

    rule1 = np.fmax(np.fmax(fuzzy_rainfall_none, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule2 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule3 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule4 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule5 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule6 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule7 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule8 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule9 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium ))
    rain_activation_no1 = np.fmin(rain_tomorrow_no, rule1)
    rain_activation_no2 = np.fmin(rain_tomorrow_no, rule2)
    rain_activation_no3 = np.fmin(rain_tomorrow_no, rule3)
    rain_activation_no4 = np.fmin(rain_tomorrow_no, rule4)
    rain_activation_no5 = np.fmin(rain_tomorrow_no, rule5)
    rain_activation_no6 = np.fmin(rain_tomorrow_no, rule6)
    rain_activation_no7 = np.fmin(rain_tomorrow_no, rule7)
    rain_activation_no8 = np.fmin(rain_tomorrow_no, rule8)
    rain_activation_no9 = np.fmin(rain_tomorrow_no, rule9)

    rain_activation_no = np.fmin(np.fmin(np.fmin(np.fmin(rain_activation_no1, rain_activation_no2), np.fmin(rain_activation_no3, rain_activation_no4)),
                        np.fmin(np.fmin(rain_activation_no5, rain_activation_no6), np.fmin(rain_activation_no7, rain_activation_no8))),
                        rain_activation_no9)

    rule10 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule11 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule12 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule13 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule14 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule15 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule16 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule17 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule18 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule19 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule20 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule21 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule22 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule23 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule24 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule25 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule26 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule27 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule28 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule29 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule30 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule31 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule32 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule33 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule34 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))

    rain_activation_yes = np.fmin(np.fmin(np.fmin(np.fmin(np.fmin(rule10, rule11), np.fmin(rule12, rule13)),
                                    np.fmin(np.fmin(rule14, rule15), np.fmin(rule16, rule17))),
                            np.fmin(np.fmin(np.fmin(rule18, rule19), np.fmin(rule20, rule21)),
                                    np.fmin(np.fmin(rule22, rule23), np.fmin(rule24, rule25)))),
                            np.fmin(np.fmin(np.fmin(np.fmin(rule26, rule27), np.fmin(rule28, rule29)),
                                    np.fmin(np.fmin(rule30, rule31), np.fmin(rule32, rule33))),
                            rule34))
    aggregated = np.fmax(rain_activation_no, rain_activation_yes)
    #print(aggregated)
    if aggregated.mean() == 0:
            #print('======================================================')
            return -10
    else:
            rain = fuzz.defuzz(rain_tomorrow, aggregated, defu)
            return rain

In [ ]:
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction_trapezoid(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print(len(predictions))
    print('pred count -10 = ', predictions.count(-10))
    pred = []
    for i in predictions:
        if i > 0.5:
            pred.append(1)
        elif i == -10:
            pred.append(3)
        else:
            pred.append(0)
    yes_l = 0
    for i in range(0,len(list(y_test)), 1):
        if list(y_test)[i] == pred[i]:
            yes_l += 1
    print(yes_l)
    print(f, '>0.5 = ', yes_l/(len(list(y_test))-predictions.count(-10))*100)
    
    
 

# Yes/no gaussian

In [229]:
def prediction_gauss(rainfall_level_m, temp_m, humidity_m, pressure_m, defu):

    rainfall_level = np.arange(0, 351, 1)
    rainfall_none = fuzz.gaussmf(rainfall_level, 0, 1.5)
    rainfall_light = fuzz.gaussmf(rainfall_level, 70, 35)
    rainfall_medium = fuzz.gaussmf(rainfall_level, 210, 35)
    rainfall_heavy = fuzz.gaussmf(rainfall_level, 340, 35)

    temp = np.arange(-10, 41, 1)
    temp_cold = fuzz.gaussmf(temp, -10, 7)
    temp_mild = fuzz.gaussmf(temp, 15, 7)
    temp_hot = fuzz.gaussmf(temp, 40, 7)

    humidity = np.arange(0, 101, 1)
    humidity_low = fuzz.gaussmf(humidity, 0, 17)
    humidity_medium = fuzz.gaussmf(humidity, 50, 17)
    humidity_high = fuzz.gaussmf(humidity, 100, 17)
    
    pressure = np.arange(980, 1061, 1)
    pressure_low = fuzz.gaussmf(pressure, 980, 15)
    pressure_medium = fuzz.gaussmf(pressure, 1020, 15)
    pressure_high = fuzz.gaussmf(pressure, 1060, 15)

    rain_tomorrow = np.arange(0, 1.1, 0.1)
    rain_tomorrow_no = fuzz.trimf(rain_tomorrow, [0, 0, 0.5])
    rain_tomorrow_yes =  fuzz.trimf(rain_tomorrow, [0.5, 1, 1])

    # Predict "raintomorrow" using the fuzzy system and linguistic variables
    fuzzy_rainfall_none = fuzz.interp_membership(rainfall_level, rainfall_none, rainfall_level_m)
    fuzzy_rainfall_light = fuzz.interp_membership(rainfall_level, rainfall_light, rainfall_level_m)
    fuzzy_rainfall_medium = fuzz.interp_membership(rainfall_level, rainfall_medium, rainfall_level_m)
    fuzzy_rainfall_heavy = fuzz.interp_membership(rainfall_level, rainfall_heavy, rainfall_level_m)
    fuzzy_temp_cold = fuzz.interp_membership(temp, temp_cold, temp_m)
    fuzzy_temp_mild = fuzz.interp_membership(temp, temp_mild, temp_m)
    fuzzy_temp_hot = fuzz.interp_membership(temp, temp_hot, temp_m)
    fuzzy_humidity_low = fuzz.interp_membership(humidity, humidity_low, humidity_m)
    fuzzy_humidity_medium = fuzz.interp_membership(humidity, humidity_medium, humidity_m)
    fuzzy_humidity_high = fuzz.interp_membership(humidity, humidity_high, humidity_m)
    fuzzy_pressure_low = fuzz.interp_membership(pressure, pressure_low, pressure_m)
    fuzzy_pressure_medium = fuzz.interp_membership(pressure, pressure_medium, pressure_m)
    fuzzy_pressure_high = fuzz.interp_membership(pressure, pressure_high, pressure_m)

    rule1 = np.fmax(np.fmax(fuzzy_rainfall_none, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule2 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule3 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule4 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule5 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule6 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule7 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule8 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule9 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium ))
    rain_activation_no1 = np.fmin(rain_tomorrow_no, rule1)
    rain_activation_no2 = np.fmin(rain_tomorrow_no, rule2)
    rain_activation_no3 = np.fmin(rain_tomorrow_no, rule3)
    rain_activation_no4 = np.fmin(rain_tomorrow_no, rule4)
    rain_activation_no5 = np.fmin(rain_tomorrow_no, rule5)
    rain_activation_no6 = np.fmin(rain_tomorrow_no, rule6)
    rain_activation_no7 = np.fmin(rain_tomorrow_no, rule7)
    rain_activation_no8 = np.fmin(rain_tomorrow_no, rule8)
    rain_activation_no9 = np.fmin(rain_tomorrow_no, rule9)

    rain_activation_no = np.fmin(np.fmin(np.fmin(np.fmin(rain_activation_no1, rain_activation_no2), np.fmin(rain_activation_no3, rain_activation_no4)),
                        np.fmin(np.fmin(rain_activation_no5, rain_activation_no6), np.fmin(rain_activation_no7, rain_activation_no8))),
                        rain_activation_no9)

    rule10 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule11 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule12 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule13 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule14 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule15 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule16 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule17 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule18 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule19 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule20 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule21 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule22 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule23 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule24 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule25 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule26 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule27 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule28 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule29 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule30 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule31 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule32 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule33 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule34 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))

    rain_activation_yes = np.fmin(np.fmin(np.fmin(np.fmin(np.fmin(rule10, rule11), np.fmin(rule12, rule13)),
                                    np.fmin(np.fmin(rule14, rule15), np.fmin(rule16, rule17))),
                            np.fmin(np.fmin(np.fmin(rule18, rule19), np.fmin(rule20, rule21)),
                                    np.fmin(np.fmin(rule22, rule23), np.fmin(rule24, rule25)))),
                            np.fmin(np.fmin(np.fmin(np.fmin(rule26, rule27), np.fmin(rule28, rule29)),
                                    np.fmin(np.fmin(rule30, rule31), np.fmin(rule32, rule33))),
                            rule34))
    aggregated = np.fmax(rain_activation_no, rain_activation_yes)
    #print(aggregated)
    if aggregated.mean() == 0:
            #print('======================================================')
            return -10
    else:
            rain = fuzz.defuzz(rain_tomorrow, aggregated, defu)
            return rain

In [ ]:
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction_gauss(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print(len(predictions))
    print('pred count 0 = ', predictions.count(0))
    print('pred count -10 = ', predictions.count(-10))
    pred = []
    for i in predictions:
        if i > 0.5:
            pred.append(1)
        elif i == -10:
            pred.append(3)
        else:
            pred.append(0)
    yes_l = 0
    for i in range(0,len(list(y_test)), 1):
        if list(y_test)[i] == pred[i]:
            yes_l += 1
    print(yes_l)
    print(f, '>0.5 = ', yes_l/(len(list(y_test))-predictions.count(-10))*100)


# Yes/no bell

In [231]:
def prediction_bell(rainfall_level_m, temp_m, humidity_m, pressure_m, defu):

    rainfall_level = np.arange(0, 351, 1)
    rainfall_none = fuzz.gbellmf(rainfall_level, 2, 5, 0)
    rainfall_light = fuzz.gbellmf(rainfall_level, 50, 2, 80)
    rainfall_medium = fuzz.gbellmf(rainfall_level, 50, 2, 215)
    rainfall_heavy = fuzz.gbellmf(rainfall_level, 50, 2, 350)
    
    temp = np.arange(-10, 41, 1)
    temp_cold = fuzz.gbellmf(temp, 10, 5, -10)
    temp_mild = fuzz.gbellmf(temp, 10, 5, 15)
    temp_hot = fuzz.gbellmf(temp, 10, 5, 40)
    
    humidity = np.arange(0, 101, 1)
    humidity_low = fuzz.gbellmf(humidity, 20, 5, 0)
    humidity_medium = fuzz.gbellmf(humidity, 20, 5, 50)
    humidity_high = fuzz.gbellmf(humidity, 20, 5, 100)
    
    pressure = np.arange(980, 1061, 1)
    pressure_low = fuzz.gbellmf(pressure, 18, 5, 980)
    pressure_medium = fuzz.gbellmf(pressure, 18, 5, 1020)
    pressure_high = fuzz.gbellmf(pressure, 18, 5, 1060)

    rain_tomorrow = np.arange(0, 1.1, 0.1)
    rain_tomorrow_no = fuzz.trimf(rain_tomorrow, [0, 0, 0.5])
    rain_tomorrow_yes =  fuzz.trimf(rain_tomorrow, [0.5, 1, 1])

    # Predict "raintomorrow" using the fuzzy system and linguistic variables
    fuzzy_rainfall_none = fuzz.interp_membership(rainfall_level, rainfall_none, rainfall_level_m)
    fuzzy_rainfall_light = fuzz.interp_membership(rainfall_level, rainfall_light, rainfall_level_m)
    fuzzy_rainfall_medium = fuzz.interp_membership(rainfall_level, rainfall_medium, rainfall_level_m)
    fuzzy_rainfall_heavy = fuzz.interp_membership(rainfall_level, rainfall_heavy, rainfall_level_m)
    fuzzy_temp_cold = fuzz.interp_membership(temp, temp_cold, temp_m)
    fuzzy_temp_mild = fuzz.interp_membership(temp, temp_mild, temp_m)
    fuzzy_temp_hot = fuzz.interp_membership(temp, temp_hot, temp_m)
    fuzzy_humidity_low = fuzz.interp_membership(humidity, humidity_low, humidity_m)
    fuzzy_humidity_medium = fuzz.interp_membership(humidity, humidity_medium, humidity_m)
    fuzzy_humidity_high = fuzz.interp_membership(humidity, humidity_high, humidity_m)
    fuzzy_pressure_low = fuzz.interp_membership(pressure, pressure_low, pressure_m)
    fuzzy_pressure_medium = fuzz.interp_membership(pressure, pressure_medium, pressure_m)
    fuzzy_pressure_high = fuzz.interp_membership(pressure, pressure_high, pressure_m)

    rule1 = np.fmax(np.fmax(fuzzy_rainfall_none, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule2 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_low))
    rule3 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule4 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule5 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule6 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_medium))
    rule7 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule8 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium))
    rule9 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_medium ))
    rain_activation_no1 = np.fmin(rain_tomorrow_no, rule1)
    rain_activation_no2 = np.fmin(rain_tomorrow_no, rule2)
    rain_activation_no3 = np.fmin(rain_tomorrow_no, rule3)
    rain_activation_no4 = np.fmin(rain_tomorrow_no, rule4)
    rain_activation_no5 = np.fmin(rain_tomorrow_no, rule5)
    rain_activation_no6 = np.fmin(rain_tomorrow_no, rule6)
    rain_activation_no7 = np.fmin(rain_tomorrow_no, rule7)
    rain_activation_no8 = np.fmin(rain_tomorrow_no, rule8)
    rain_activation_no9 = np.fmin(rain_tomorrow_no, rule9)

    rain_activation_no = np.fmin(np.fmin(np.fmin(np.fmin(rain_activation_no1, rain_activation_no2), np.fmin(rain_activation_no3, rain_activation_no4)),
                        np.fmin(np.fmin(rain_activation_no5, rain_activation_no6), np.fmin(rain_activation_no7, rain_activation_no8))),
                        rain_activation_no9)

    rule10 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule11 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule12 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule13 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule14 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule15 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule16 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule17 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule18 = np.fmax(np.fmax(fuzzy_rainfall_light, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule19 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule20 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule21 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule22 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule23 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule24 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule25 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule26 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule27 = np.fmax(np.fmax(fuzzy_rainfall_medium, fuzzy_temp_hot), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule28 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_low, fuzzy_pressure_high))
    rule29 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule30 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule31 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_cold), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))
    rule32 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_medium, fuzzy_pressure_high))
    rule33 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_medium))
    rule34 = np.fmax(np.fmax(fuzzy_rainfall_heavy, fuzzy_temp_mild), np.fmax(fuzzy_humidity_high, fuzzy_pressure_high))

    rain_activation_yes = np.fmin(np.fmin(np.fmin(np.fmin(np.fmin(rule10, rule11), np.fmin(rule12, rule13)),
                                    np.fmin(np.fmin(rule14, rule15), np.fmin(rule16, rule17))),
                            np.fmin(np.fmin(np.fmin(rule18, rule19), np.fmin(rule20, rule21)),
                                    np.fmin(np.fmin(rule22, rule23), np.fmin(rule24, rule25)))),
                            np.fmin(np.fmin(np.fmin(np.fmin(rule26, rule27), np.fmin(rule28, rule29)),
                                    np.fmin(np.fmin(rule30, rule31), np.fmin(rule32, rule33))),
                            rule34))
    aggregated = np.fmax(rain_activation_no, rain_activation_yes)
    #print(aggregated)
    if aggregated.mean() == 0:
            #print('======================================================')
            return -10
    else:
            rain = fuzz.defuzz(rain_tomorrow, aggregated, defu)
            return rain

In [ ]:
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction_bell(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print(len(predictions))
    print('pred count -10 = ', predictions.count(-10))
    pred = []
    for i in predictions:
        if i > 0.5:
            pred.append(1)
        elif i == -10:
            pred.append(3)
        else:
            pred.append(0)
    yes_l = 0
    for i in range(0,len(list(y_test)), 1):
        if list(y_test)[i] == pred[i]:
            yes_l += 1
    print(yes_l)
    print(f, '>0.5 = ', yes_l/(len(list(y_test))-predictions.count(-10))*100)

    

In [265]:
X = df.drop(['RainToday'], axis=1)
y = df['RainToday']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    start = dt.now()
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print('time = ', dt.now() - start)
print('trapezoid')
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    start = dt.now()
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction_trapezoid(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print('time = ', dt.now() - start)
print('gauss')
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    start = dt.now()
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction_gauss(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print('time = ', dt.now() - start)
print('bell')
for f in ['centroid', 'bisector', 'mom', 'som', 'lom']:
    start = dt.now()
    print(f)
    predictions = []
    pred_bin = []
    for index, row in X_test.iterrows():
        #print(row['Rainfall'], row['Temp9am'], row['Humidity9am'], row['Pressure9am'])
        predf = prediction_bell(row['Rainfall']+0.000000001, row['Temp9am'], row['Humidity9am'], row['Pressure9am'], f)
        predictions.append(predf)
    print('time = ', dt.now() - start)
        

centroid
time =  0:00:40.968492
bisector
time =  0:00:39.372683
mom
time =  0:00:36.140818
som
time =  0:00:39.210173
lom
time =  0:00:41.133699
trapezoid
centroid
time =  0:01:20.222379
bisector
time =  0:01:16.921489
mom
time =  0:01:28.975713
som
time =  0:01:34.085599
lom
time =  0:01:27.829425
gauss
centroid
time =  0:00:34.626456
bisector
time =  0:00:27.922777
mom
time =  0:00:25.299892
som
time =  0:00:23.241836
lom
time =  0:00:24.376917
bell
centroid
time =  0:00:28.354154
bisector
time =  0:00:29.848550
mom
time =  0:00:29.822888
som
time =  0:00:26.152586
lom
time =  0:00:32.744019


# Decision tree

In [267]:
from sklearn.tree import DecisionTreeClassifier
#split the data into training and testing sets
start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  72.0 %
Time:  0:00:00.450695


# LogisticRegression

In [260]:
from sklearn.linear_model import LogisticRegression

start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  79.0 %
Time:  0:00:00.408369


# GaussianNB

In [261]:
from sklearn.naive_bayes import GaussianNB

start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = GaussianNB()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  79.0 %
Time:  0:00:00.040986


# LinearDiscriminantAnalysis

In [262]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  80.0 %
Time:  0:00:00.115003


# KNeighborsClassifier

In [263]:
from sklearn.neighbors import KNeighborsClassifier

start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = KNeighborsClassifier()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  78.0 %
Time:  0:00:00.251769


# SVC

In [268]:
from sklearn.svm import SVC

start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = SVC()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  78.0 %
Time:  0:06:33.543984


# RandomForestClassifier

In [269]:
from sklearn.ensemble import RandomForestClassifier

start = dt.now()
X = df[['Rainfall',  'Humidity9am', 'Pressure9am','Temp9am']]
y = df['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
time = dt.now() - start
#calculate accuracy
accuracy=model.score(X_test,y_test)
print('Accuracy: ',round(accuracy,2)*100,'%')
print('Time: ', time)

Accuracy:  78.0 %
Time:  0:00:13.238590
